## 動的計画法
全状態を探索すると計算時間が膨大になるから（同じ計算を繰り返すことによる）、一度計算した部分を保存しておいて再利用（メモ化）
どんな問題も無限の資源があれば全組み合わせを試して解くことができるが、それらの中には部分的に同じものがあるので、そういった繰り返しを避けられるのがメリット  
DPのDはDynamic ArrayのDか  

In [17]:
#01
"""
引数の組は高々品物の数と重さの積通り
i番目以降の品物から選んだ時、重さがjを超えない最大の価値
品物を詰めるたびにjをその品の重さだけ減らす
よくある方法では、i番目まで選べるとき重さがjを超えない最大価値みたいにDPを組むが、
やってること自体にそんなに差はない。DPはいろんな置き方があるのでいろいろできるといいか。
でもいつもの２重forのほうが分かりやすい

"""
N,W = map(int, input().split())
wv = [list(map(int, input().split())) for _ in range(N)]

dp = [[-1 for _ in range(W+1)] for _ in range(N+1)]

# i番目以降から重さの総和がｊ以下になるよう選ぶ
def solve(i,j):
    # 探索済み
    print("--", i,j)
    if dp[i][j] >= 0:
        return dp[i][j]
    res = 0
    # 全部調べた
    if i == N:
        res = 0
    # i番目のwが制限より大きいので選べない
    elif  j < wv[i][0]:
        # 次以降の品をみる
        res = solve(i+1, j)
    else:
        # i番目を入れる場合と入れない場合の両方の大きいほうについてみて大きいほうを選ぶ
        res = max(solve(i+1, j), solve(i+1, j - wv[i][0]) + wv[i][1])
    dp[i][j] = res
    return dp[i][j]

solve(0,W)
print(dp[0][W])
print(dp)

4 5
2 3
1 2
3 4
2 2
-- 0 5
-- 1 5
-- 2 5
-- 3 5
-- 4 5
-- 4 3
-- 3 2
-- 4 2
-- 4 0
-- 2 4
-- 3 4
-- 4 4
-- 4 2
-- 3 1
-- 4 1
-- 1 3
-- 2 3
-- 3 3
-- 4 3
-- 4 1
-- 3 0
-- 4 0
-- 2 2
-- 3 2
7
[[-1, -1, -1, -1, -1, 7], [-1, -1, -1, 4, -1, 6], [-1, -1, 2, 4, 4, 6], [0, 0, 2, 2, 2, 2], [0, 0, 0, 0, 0, 0]]


In [23]:

"""
２重for ver
漸化式でやるやつ
個人的にこっちの方がメモ化より好き
"""
N,W = map(int, input().split())
wv = [list(map(int, input().split())) for _ in range(N)]

dp = [[0 for _ in range(W+1)] for _ in range(N)]

for i in range(N):
    for j in range(W+1):
        # i番目の品の重さが制限を超えるなら選べない
        if wv[i][0] > j:
            if i == 0:
                dp[i][j] = 0
            else:
                dp[i][j] = dp[i-1][j]
        else:
            # i番目を選んだ場合と選ばなかった場合の大きい方
            dp[i][j] = max(dp[i-1][j], dp[i-1][j - wv[i][0]] + wv[i][1])
            
print(dp[N-1][W])
print(dp)

4 5
2 3
1 2
3 4
2 2
7
[[0, 0, 3, 3, 3, 3], [0, 2, 3, 5, 5, 5], [0, 2, 3, 5, 6, 7], [0, 2, 3, 5, 6, 7]]


## 最長共通部分列（LCS）問題
典型問題なのでマスターしたい。  
dp(i,j)：Sのi文字目までとTのj文字目までのLCSの長さ  
とおくと、  
S(i)=T(j)：既存のLCSにS(i+1)を加えたもの  
あるいは  
Sのi+1文字目までとTのj文字目までのLCSの長さ  
か  
Sのi文字目までとTのj+1文字目までのLCSの長さ  
が最長になる  
if S(i) == T(j):  
    dp(i+1,j+1) = dp(i,j) + 1  
else:  
    dp(i+1,j+1) = max( dp(i+1,j), dp(i,j+1) )  
      
みたいな  
back trackingによる復元とかもあるがこれもちょっと難しい  

In [2]:
S = input()
T = input()

#dp[i+1][j+1]：sのi文字目までとdのj文字目まで見たときのLCSの長さ
dp = [[0 for i in range(len(T)+1)] for _ in range(len(S)+1)]

# それぞれの先頭から一文字ずつ増やしたときのLCSの長さを確認していく
# len(S)*len(T)
for i in range(len(S)):
    for j in range(len(T)):
        if S[i] == T[j]:
            dp[i+1][j+1] = dp[i][j] + 1
        else:
            dp[i+1][j+1] = max(dp[i][j+1], dp[i+1][j])
            
print(dp[len(S)][len(T)])

abcd
becd
3


In [ ]:
## 制限なしナップザック問題
今まで：i番目までの商品を選べる時、jまで鞄に入れられるなら    
制限なし：上に加えて、それぞれk個までいれるかも  
→i*j*kの３重ループ  
  
問題は一種類につき何個でも選べること。  
dp[i][j]:dp[i][j-w]+v[i]で、一個ずつ増やすみたいな形でkのループを消す。

In [4]:
N = int(input())
W = int(input())
wv = [list(map(int, input().split())) for _ in range(N)]

dp = [[0 for _ in range(W+1)] for _ in range(N)]

# 商品一つ目からN個目まで
for i in range(N):
    # 0からWまで
    for j in range(W+1):
        # 制限を超える場合はその商品を選べないのでその商品を選ばない状態での最大を選択
        if wv[i][0] > j:
            if i == 0:
                dp[i][j] = 0
            else:
                dp[i][j] = dp[i-1][j]
        else:
            # i番目の価値が小さいとi-1番目までで作る方が価値が高い場合があるのでちゃんと比較
            # ここもi=0かどうかでやらないといけないのでこの書き方はまずい
            dp[i][j] = max(dp[i-1][j], dp[i][j - wv[i][0]] + wv[i][1])
            
print(dp[N-1][W])

3
7
3 4
4 5
2 3
10


## N*Wが大きいナップザック問題
こういう場合は重さに対する価値を最大にするのではなく、価値に対する重さを最小にするとうまくいく  
*最小と最大は切り替えられる*  
dp(i,j):価値の総和がjになるように選んだ時の最小の重さ


In [19]:
N = int(input())
W = int(input())
INF = float("inf")
wv = [list(map(int, input().split())) for _ in range(N)]
V = 100
dp = [[INF for _ in range(N*V + 1)] for _ in range(N+1)]
dp[0][0] = 0

for i in range(N):
    for j in range(N*V+1):
        
        if j < wv[i][1]:
            # i番目の価値がjを超えるので選べない
            dp[i+1][j] = dp[i][j]
        else:
            dp[i+1][j] = min(dp[i][j], dp[i][j - wv[i][1]] + wv[i][0])

            
# N個全部選べる時の重さがWを超えない最大価値を探す
# 価値はインデックスに対応していて要素がその時の最小重さなので注意
ans = 0
for i in range(len(dp[N])):
    if dp[N][i] <= W:
        ans = i
    
        
print(ans)
print(dp[N])

4
5
2 3
1 2
3 4
2 2
7
[0, inf, 1, 2, 3, 3, 4, 5, 6, 6, inf, 8, inf, inf, inf, inf, inf, inf, inf, inf, inf]


In [2]:
N,W = map(int, input().split())
wv = [list(map(int, input().split())) for _ in range(N)]
INF = float("inf")
V = 100 # 価値の最大値
dp = [[ INF for _ in range(N*W+1)] for _ in range(N+1)]
dp[0][0] = 0

for i in range(N):
    for j in range(N*W+1):
        if wv[i][1] > j:
            continue
        else:
            dp[i+1][j] = min(dp[i][j], dp[i][j - wv[i][1]] + wv[i][0])

ans = 0
for i in range(len(dp[N])):
    if dp[N][i] <= W:
        ans = i
        
print(ans)

4 5
2 3
1 2
3 4
2 2
7


## 個数制限付きナップザック


dp(i+1,j):i番目まででｊを作る際に余るi番目の個数（作れないなら-1）  

i-1まででｊができるならまるまるiを残せる。j-a_iができるなら（そしてそこまででa_iが一つでも残ってるなら）jができる、みたいに考えていける。ｊを０から増やしていって判断。a_iを残せる個数が漸化式の値になるのは変な感じもするが、値が負でなければjを作れる→漸化式の値でなくインデックスも利用して情報を渡していくみたいな？？？

In [5]:
N = int(input())
A = list(map(int, input().split()))
M = list(map(int, input().split()))
K = int(input())

dp = [[-1 for _ in range(K+1)] for _ in range(N+1)]
dp[0][0] = 0

for i in range(N):
    for j in range(K+1):
        # a_iなしでｊができるので丸々残せる
        # elifの条件より後にしてしまうとi-1番目まででｊ作れるのにそれを使わないことになるので注意
        if dp[i][j] >= 0:
            dp[i+1][j] = M[i]
            
        # a_iがそもそもｊを超えているor j - a_iを作るとa_iが足りなくなる
        elif A[i] > j or dp[i+1][j - A[i]] <= 0:
            continue
        # j - a_i　に a_iを一つ足してｊを作るので、a_iの余り個数が一つ減る
        else:
            dp[i+1][j] = dp[i+1][j - A[i]] - 1
            
print(dp[N][K] >= 0)
print(dp)

"""
バックトラッキングをやるなら、
dp[N][K]を見る→N番目が何個余るか（N番目を何個使うか）がわかる
→dp[N-1][K-a_N-1*N-1番目を使った数]を見る→N-1番目が何個余るか（N-1番目を何個使うか）がわかる
→・・・
で何を何個使うかを調べられるはず
"""

3
3 5 8
3 2 2
17
True
[[0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], [3, -1, -1, 2, -1, -1, 1, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1], [2, -1, -1, 2, -1, 1, 2, -1, 1, 2, 0, 1, -1, 0, 1, -1, 0, -1], [2, -1, -1, 2, -1, 2, 2, -1, 2, 2, 2, 2, -1, 2, 2, -1, 2, 1]]


## 最長増加部分列

dp(i)：最後がa_iであるような最長の増加部分列の長さ  
とすると、そのような増加部分列は  
a_iのみか、それまでの最長増加部分列にa_iを付け足したもの  
となるので、  
dp(i) = max(1, dp(j)+1)   (j<i でa_j < a_iの時後者)  
これはO(N**2)(i,j　で二重for回す)
  
最小部分列の最後尾の値が小さいほど続けやすいので、  
dp(i)：長さがiの最小部分列の最終要素の値の最小値（なければINF）    
でもできる


In [1]:
from bisect import bisect_left
N = int(input())

A = list(map(int, input().split()))

INF = float("inf")
dp = [INF] * N



for i in range (N):
    # 一つ目はそのまま最長増加部分列になる
    if i == 0:
        dp[i] = A[i]
    else:
        # 最長増加部分列の現時点の最後のやつより小さい値は、どこか知ら入れる場所（更新できるところ）があるのでbisectで探して入れる
        if dp[i-1] > A[i]:
            idx = bisect_left(dp, A[i])
            dp[idx] = A[i]
        # 最長増加部分列の最後より大きい値が出てきたらそれの末尾に追加できる
        elif dp[i-1] < A[i]:
            dp[i] = A[i]
            
ans = 0
for i in range(N):
    if dp[i] != INF:
        ans += 1
        
print(ans)
print(dp)

"""
input

5
4 2 3 1 5
"""

5
4 2 3 1 5
3
[1, 3, 5, inf, inf]


## 分割数
http://drken1215.hatenablog.com/entry/2018/01/16/222843  

P(n,k)：nをk個以下の１以上の自然数の和で表したときの場合の数  
とおくと、  
P(n,k) = P(n-k,k) + P(n,k-1)


### 一項目の導出
１項目は０を含まないｋ個の自然数でｎを表す場合にあたる。  
Pの条件が、１以上の自然数の和で表す、なので、まず先にk個分１を割り振っておいて、そのあと残りのn-kをどのように割り振るか決める。これがP(n-k,k)


### 二項目の導出
こちらは０を含む場合を表す。ｋ個の自然数（０を含む）でｎを作るとき、ある一つのゼロを取り除いたら、k-1個の自然数でｎを作っているのと同じになる。ということで、P(n,k-1)


何個以上でもよい１以上への分割ならP(n,n)で。こいつはO(n√n)で求まるらしい



In [10]:
N,K= map(int, input().split())
M = 10**9 + 7

dp = [[0 for _ in range(K+1)] for _ in range(N+1)]
# 
dp[0][0] = 1
# iをj個に分割
for i in range(N+1):
    for j in range(K+1):
        # 分割できない場合、j-1個に分割するときと同じ
        if i < j:
            dp[i][j] = dp[i][j-1]
        else:
            dp[i][j] = dp[i-j][j] + dp[i][j-1]
            
            
print(dp[-1][-1] % M)
print(dp)

7 7
15
[[1, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 2, 2, 2, 2, 2, 2], [0, 1, 2, 3, 3, 3, 3, 3], [0, 1, 3, 4, 5, 5, 5, 5], [0, 1, 3, 5, 6, 7, 7, 7], [0, 1, 4, 7, 9, 10, 11, 11], [0, 1, 4, 8, 11, 13, 14, 15]]


In [ ]:
## 重複組み合わせ
#https://emtubasa.hateblo.jp/entry/2018/08/29/161456
難しいので復習ちゃんとする
n種類の品、i番目はa_i個ある、これらからｍ個えらぶ場合の数

同じ種類のものは区別できない

dp(i+1,j)：i番目までからj個選ぶ場合の数  
とすると、i-1までからj-k、i番目からk個選べばよい。 -> dp(i+1,j) = sigma(dp(i, j-k))  （k:0～j）
ここで、i-1までからj-k個えらぶdp(i, j-k)は、j-kをj-k-1にずらしてシグマの外で帳尻を合わせることを考える。

ずらす前にあったやつ：k=0の時のdp(i,j)を加える
ずらして出てきたやつ：k=j(かa_iの小さい方)のdp(i, j-min(j, a_i)-1)を取り除くことで、

dp(i+1,j) = sigma(dp(i, j-k)) = sigma(dp(i, j-k-1))   + dp(i,j) - dp(i, j-min(j, a_i)-1)  

ここで、右辺の１項目について考える。  
k=j (min(j, a_i) = j)のとき、dp(i, j-j-1)=0  
k=a_iのとき、j > a_iなのでk=jとならない  
よってｊをj-1に置き換えてもよく、そうすると１項目は  dp(i+1, j-1)となる

In [16]:
N,M = map(int, input().split())
A = list(map(int, input().split()))

mod = 10**9+7

dp = [[0 for _ in range(M+1)] for _ in range(N+1)]

for i in range(N+1):
    #i番目を一つも選ばない方法は一種類のみ
    dp[i][0] = 1
    
for i in range(N):
    # j = 0を上で作ったので飛ばすかたち
    for j in range(1, M+1):
        # 上のメモの３項めが０以下の場合は考慮しない
        if j - A[i] -1 < 0:
            dp[i+1][j] = dp[i+1][j-1] + dp[i][j]
        else:
            dp[i+1][j] = dp[i+1][j-1] + dp[i][j] - dp[i][j - A[i] - 1]
            print(i,j,j - A[i] - 1)
            
print(dp[-1][-1])
print(dp)

3 3
1 2 3
0 2 0
0 3 1
1 3 0
6
[[1, 0, 0, 0], [1, 1, 0, 0], [1, 2, 2, 1], [1, 3, 5, 6]]


以下類題

In [ ]:
"""
Typical DP Contest A

典型的なら制限内で最大の～とかになりそうだが、これは得点の個数。
i番目の問題までで作れる得点をi+1に移しつつ、i+1番目の得点ををそれぞれの点に加える。
よくあるdpぽい式にしたが、キューを使ってもうまくいくと思う（i番目まででできた得点をキューに持っておいて、i+1番目の点をそれぞれに足して新たな点を作るみたいな）
"""


N = int(input())

p = list(map(int, input().split()))



dp = [[0 for _ in range(sum(p)+1)] for _ in range(N+1)]
dp[0][0] = 1

for i in range(N):
    for j in range(sum(p)+1):
        # i番目の問題まででｊ点が作れるならdp[i][j] = 1
        if dp[i][j] == 1:
            dp[i+1][j] = 1 # ここでi番目の時の情報がi+1に引き継がれる
            dp[i+1][j+p[i]] = 1


print(sum(dp[-1]))

In [ ]:
"""
Typical DP Contest D

愚直に全探索すると絶対に間に合わない
i-1回さいころを振った時に手持ちの素因数2,3,5がそれぞれa,b,c個ある確率をdp[i][a][b][c]として、
i回目のさいころの目が2だったらそれぞれをa+1,b,c個持つことになり、
その場合の数dp[i+1][a+1][b][c]はdp[i][a][b][c]を利用して求まる、みたいなことを繰り返す

TODO：結構難しかったのでまたやる
"""

from collections import Counter


def main():
    N,D = map(int, input().split())

    num = D
    i = 2
    tmp = []
    for i in [2,3,5]:
        while num % i == 0:
            num = num / i
            tmp.append(i)

    if num > 1:
        print(0)
        exit()

    cntr = Counter(tmp)

    #さいころを投げて、2の素因数がa,3の～がｂ、５の～がｃ個ある場合の数 (i回さいころをふった時に)
    dp = [[[[0.0 for _ in range(cntr[5]+1)] for _ in range(cntr[3]+1)] for _ in range(cntr[2]+1)] for _ in range(N+1)]
    dp[0][0][0][0] = 1.0

    # 2がcntr[2]回以上出る場合の確率は全部cntr[2]の所に入れる
    # 最終的にDの倍数となる確率が分かればいいので。
    # でないと最後にそれぞれをcntr[2],cntr[3],cntr[5]個以上含むDの倍数になる確率を全て足し合わせる必要がある
    for i in range(N):
        for a in range(cntr[2]+1):
            for b in range(cntr[3]+1):
                for c in range(cntr[5]+1):
                    # 以下１～６までが出たときのdpの変化

                    # １が出ても何も変わらない
                    dp[i+1][a][b][c] += dp[i][a][b][c]/6.0
                    # ２が出たらaが一つ増える感じ
                    dp[i+1][min(a+1, cntr[2])][b][c] += dp[i][a][b][c]/6.0
                    # 3
                    dp[i+1][a][min(b+1, cntr[3])][c] += dp[i][a][b][c]/6.0
                    # 4は２が２個なので
                    dp[i+1][min(a+2, cntr[2])][b][c] += dp[i][a][b][c]/6.0
                    # 5
                    dp[i+1][a][b][min(c+1, cntr[5])] += dp[i][a][b][c]/6.0
                    # 6は２と３が一個ずつ
                    dp[i+1][min(a+1, cntr[2])][min(b+1, cntr[3])][c] += dp[i][a][b][c]/6.0

    print(dp[-1][cntr[2]][cntr[3]][cntr[5]])

if __name__ == "__main__":
    main()